In [34]:
import random 
import csv
import os

In [35]:
class Store:
    def __init__(self, store_id, store_name, branch, estimated_bags, rating, longitude, latitude):
        # private attributes
        self._store_id = store_id
        self._store_name = store_name
        self._branch = branch 
        self._rating = rating 
        self._price_per_bag = None 
        self._estimated_bags = estimated_bags # average_bags_at_9AM
        self._actual_bags = None # actual bags at night
        self._bags_reserved = 0 
        self._bags_canceled = 0 
        self._bags_sold = 0 
        self._longitude = longitude
        self._latitude = latitude

        # WHAT WE MIGHT ADD LATER:
        # cancellation rate history
        # location

    # setters
    def set_price_per_bag(self, price):
        self._price_per_bag = price 

    def set_actual_bags(self, count):
        self._actual_bags = count 


    # getters
    def get_store_id(self):
        return self._store_id
    
    def get_store_name(self):
        return self._store_name
    
    def get_branch(self):
        return self._branch
    
    def get_price_per_bag(self):
        return self._price_per_bag 

    def get_estimated_bags(self):
        return self._estimated_bags 

    def get_actual_bags(self):
        return self._actual_bags 
    
    def get_unreserved_bags(self):
        # use actual bags if we have them, otherwise use estimated
        total_bags = self._actual_bags if self._actual_bags is not None else self._estimated_bags
        # how many bags are still free to be reserved
        return max(0, total_bags - self._bags_reserved)
    
    def get_rating(self):
        return self._rating
    
    def get_location(self):
        return {self._longitude, self._latitude}

    
    def reserve_bag(self):
        # when a bag is reserved, we also count it as sold in this simple model
        self._bags_reserved += 1
        self._bags_sold += 1

    def cancel_bag(self):
        # cancel one reserved bag and fix the counters
        self._bags_canceled += 1
        self._bags_reserved = max(0, self._bags_reserved - 1)
        self._bags_sold = max(0, self._bags_sold - 1)

    def sell_bag(self):
        self._bags_sold += 1

In [36]:
class Customer:
    def __init__(self, customer_id, valuations, longitude, latitude):
        self._customer_id = customer_id
        self._valuations = valuations
        self._longitude = longitude
        self._latitude = latitude
        self._reserved_store_ID = None # store the store at which the customer reserved a bag from for future purposes (and printing purposes too)


    def get_customer_id(self):
        return self._customer_id
    
    def get_valuation(self, store_id):
        return self._valuations[store_id]
    
    def get_location(self):
        return {self._longitude, self._latitude}
    
    def get_reserved_store(self):
        return self._reserved_store_ID
    
    def reserve(self, store_id):
        self._reserved_store_ID = store_id


    

In [37]:
class Marketplace:
    def __init__(self, n=10, stores=None, customers=None):
        # how many stores we show to each customer
        self._n = n

        # list of Store objects in the system
        self._stores = stores if stores else []

        # list of Customer objects in the system
        self._customers = customers if customers else []

        self.algorithm1 = Algorithm1()

        # for each store: how many times it was shown to customers
        # used later for fairness (exposure) KPI
        self._exposure_counts = {}
        for store in self._stores:
            self._exposure_counts[store.get_store_id()] = 0

    def add_store(self, store):
        # add a new store and start its exposure count at 0
        self._stores.append(store)
        self._exposure_counts[store.get_store_id()] = 0

    def add_customer(self, customer):
        # add a new customer to the system
        self._customers.append(customer)

    def get_stores(self):
        return self._stores

    def get_customers(self):
        return self._customers

    def set_stores(self, stores):
        # replace all stores (used when we load from dataset)
        self._stores = stores
        # reset exposure counts for the new list of stores
        self._exposure_counts = {store.get_store_id(): 0 for store in self._stores}

    def set_customers(self, customers):
        self._customers = customers

    def select(self, customer):
        """
        main selection function used in milestone 2
        we use a greedy baseline: show top-N stores by rating that still have bags
        """
        selected_stores = self._greedy_baseline(self._stores, self._n)

        # we record that these stores were shown to this customer
        self._record_exposure(selected_stores)

        return selected_stores

    def _greedy_baseline(self, stores, N):
        """
        choose at most N stores:
        1) they must still have unreserved bags
        2) we sort them by rating (highest first)
        """
        available_stores = []

        for store in stores:
            # get_unreserved_bags() = how many bags are still free to sell
            if store.get_unreserved_bags() > 0:
                available_stores.append(store)

        # sort by rating in descending order
        available_stores.sort(key=lambda s: s.get_rating(), reverse=True)

        # return only the first N stores (or fewer if list is smaller)
        return available_stores[:N]

    def _record_exposure(self, selected_stores):
        # every time a store is shown, we increase its exposure count by 1
        for store in selected_stores:
            sid = store.get_store_id()
            if sid not in self._exposure_counts:
                self._exposure_counts[sid] = 0
            self._exposure_counts[sid] += 1

    def compute_kpis(self):
        """
        compute KPIs for one simulation run (one "day")
        we aggregate over all stores and all customers
        """
        total_reserved = 0
        total_canceled = 0
        total_actual_bags = 0
        total_sold_bags = 0

        total_revenue = 0.0
        max_possible_revenue = 0.0

        for store in self._stores:
            # price per bag (if not set, we treat it as 0 to avoid errors)
            price = store.get_price_per_bag()
            if price is None:
                price = 0.0

            # estimated bags from 9AM (given in the dataset)
            estimated = store.get_estimated_bags()

            # if actual bags are not implemented, we approximate with estimated
            actual = store.get_actual_bags() if hasattr(store, "get_actual_bags") else None
            if actual is None:
                actual = estimated

            # these counters are stored inside Store
            reserved = store._bags_reserved
            canceled = store._bags_canceled

            # sold bags: in our simple model, we use sold or (reserved - canceled)
            sold = store._bags_sold
            if sold == 0:
                sold = max(0, reserved - canceled)

            total_reserved += reserved
            total_canceled += canceled
            total_actual_bags += actual
            total_sold_bags += sold

            # money we actually made
            total_revenue += sold * price
            # money we could have made if we sold all actual bags
            max_possible_revenue += actual * price

        # unsold bags = food that is still wasted
        unsold_bags = max(0, total_actual_bags - total_sold_bags)

        # if there were reservations, we can compute cancellation rate
        if total_reserved > 0:
            cancellation_rate = total_canceled / total_reserved
        else:
            cancellation_rate = 0.0

        # fraction of bags that ended up unsold
        if total_actual_bags > 0:
            food_waste_rate = unsold_bags / total_actual_bags
        else:
            food_waste_rate = 0.0

        # how close we are to "ideal" revenue
        if max_possible_revenue > 0:
            revenue_efficiency = total_revenue / max_possible_revenue
        else:
            revenue_efficiency = 0.0

        # money we lost because we didn't sell everything
        revenue_lost = max_possible_revenue - total_revenue

        # count customers who left without reserving any bag
        customers_left = 0
        for customer in self._customers:
            # we assume Customer has get_reserved_store(), None means no reservation
            if customer.get_reserved_store() is None:
                customers_left += 1

        # fairness of exposure between stores (1 = very fair, 0 = unfair)
        fairness_score = self._compute_fairness_score()

        # we return a dictionary so it's easy to print or plot
        return {
            "total_bags_sold": total_sold_bags,
            "total_bags_unsold": unsold_bags,
            "total_bags_canceled": total_canceled,
            "total_revenue": total_revenue,
            "max_possible_revenue": max_possible_revenue,
            "revenue_lost": revenue_lost,
            "cancellation_rate": cancellation_rate,
            "food_waste_rate": food_waste_rate,
            "revenue_efficiency": revenue_efficiency,
            "customers_left": customers_left,
            "fairness_score": fairness_score,
        }

    def _compute_fairness_score(self):
        """
        compute a simple fairness score based on exposure:
        1 - Gini(exposure)
        """
        exposures = list(self._exposure_counts.values())
        n = len(exposures)

        # no stores → we can just say fairness is 1
        if n == 0:
            return 1.0

        # if all stores were never shown, exposure is equal (all zero)
        if all(e == 0 for e in exposures):
            return 1.0

        exposures.sort()
        total = sum(exposures)
        if total == 0:
            return 1.0

        # standard Gini calculation
        cumulative = 0
        for i, x in enumerate(exposures, start=1):
            cumulative += i * x

        gini = (2 * cumulative) / (n * total) - (n + 1) / n

        # fairness score in [0, 1]
        fairness_score = max(0.0, 1.0 - gini)
        return fairness_score


In [38]:
class Algorithm1:
    def __init__(self):
        pass


    def select(self, stores, n, customer):
        selected_stores = []
        # First, check the stores that the customer rated 3+ that still have unreserved bags today
        liked_stores = self._get_high_valuation_stores(self, stores, customer) 
        
        selected_stores += liked_stores

        # IN ALL THE FUNCTIONS, WE ONLY CONSIDER STORES THAT HAVE UNRESERVED BAGS 


        # if number of liked stores is exactly n, then we just return those (selection function already sorts them)
        if len(liked_stores) == n:
             return selected_stores
        
        # if number of liked stores = 0 (for example, none of the stores they liked still has unreserved or its their first time using the app)
        # we use the main selection function (we will call it the general function) to select based on store rating, price, location and thats it
        if len(liked_stores) == 0:
            selected_stores = self._select_general(self, n, stores, customer)
            return selected_stores
            

        # if the number of liked stores < n, we also use the general function to fill the rest of n
        # we get x = (n - number of liked stores)
        # we select x those based on store rating, price, location
        if len(liked_stores) < n:
            x = self._n - len(liked_stores)
            selected_stores += self._select_general(self, x, stores, customer)
            return selected_stores
        

        # final scenario: if number of liked stores exceeds n, we use a function that takes into consideration the customer's valuation
        # we will call this function the valued selection. It is different than the general function in that it additionally uses valuation 
        # (as well as store rating, price, location)
        # for now, we will only consider the subarray of stores they liked
        # FLAW HERE!!! in this case user will not get the chance to try new stores
        if len(liked_stores) > n:
            selected_stores = self._select_valued(self, n, customer, liked_stores)
            return selected_stores


    def _get_high_valuation_stores(self, stores, customer):
            # first we have an empty array
            # for each store in self._stores
                # if store.get_unreserved_bags() > 0
                # if customer.get_valuation(store) >= 3
                    # add store to array

            # sort those stores based on either price, location, or valuation (need to decide on this)
            # return array
            pass


    def _select_general(self, x, stores, customer):
        # Implement an algorithm using these factors:
            # store.get_rating() for each store in self._stores array
            # store.get_price_per_bag() for each store in self._stores array
            # store.get_location() for each store in self._stores array (returns {longitude, latitude})

        # dont forget to first check if a store has store.get_unreserved_bags() > 0 before considering it
        pass 
        

In [39]:
class Algorithm2: # menna's strategy
    pass


In [40]:
class Algorithm3: # nadine's strategy
    pass

In [41]:
class Algorithm4: # ziad's strategy
    pass

In [42]:
class DataGenerator:
    def __init__(self):
        pass

    def _store_generator(self, n, save_file):
        try:
            STORE_NAMES = [ # stores to choose from (100 options)
            'TBS', 'Dunkin', 'Costa Coffee', 'Cilantro', 'Beano\'s', 'Paul', 'Krispy Kreme',
            'Cinnabon', 'Auntie Anne\'s', 'Domino\'s', 'Cake Cafe', 'L\'Aroma', 'Sip Coffee',
            'Coffeeshop Company', 'Brown Nose', 'Ovio', 'Brioche Dorée', 'Espresso Lab',
            'Drip', 'Koffee Kulture', 'Greco', 'Qahwa', 'Espresso House', 'Daily Dose',
            'Café Corniche', 'Bakehouse', 'The Greens', 'Flavour Republic', 'One Oak', 'Black Horse',
            'Caribou Coffee', 'Tim Hortons', 'Peet\'s Coffee', 'Second Cup', 'Gloria Jean\'s',
            'Pink Palms', 'Coffee Bean', 'Tchibo', 'Lavazza', 'Illy', 'Segafredo', 'Nero Caffè',
            'Pret A Manger', 'Joe & The Juice', 'Blue Bottle', 'Chickin Worx', 'Philz Coffee',
            'Dutch Bros', 'Coffee Republic', 'Coffee #1', 'Harris + Hoole', 'Caffè Ritazza',
            'Zooba', 'Mandarine Koueider', 'Sedra', 'Tseppas', 'Casper & Gambini\'s',
            'La Poire', 'Mori Sushi', 'Andrea Mariouteya', 'Sequoia', 'Bird Cage',
            'Osmow\'s', 'Lucille\'s', 'The Bakery', 'Pottery Cafe', 'Tabali',
            'Arabica', 'L\'Aubergine', 'Kazoku', 'Left Bank', 'Maison Thomas',
            'Mince', 'Cafe', 'Piano Piano', 'Séquence', 'Stage One',
            'Tabasco', 'Zooba', 'Fasahet Soumaya', 'Al Masryeen', 'Felfela',
            'Abou El Sid', 'Kazaz', 'Taboula', 'Deli 29', 'Street 9',
            'Mozzarella', 'Le Deck', 'Caviar & Bull', 'Crimson', 'Bua Khao',
            'Sachi', 'Kaito', 'Sangria', 'Salt', 'Stiletto',
            'LuckyEgypt', 'Momochi', 'Aperitivo', 'Birdcage', 'The Tap',
            'Pub 28', 'Cairo Kitchen', 'Caribou Coffee', 'Pier 88', 'Moishi',
            'Hard Rock Cafe', 'Texas Roadhouse', 'Buffalo Burger', 'Mo\'men', 'Cook Door'
            ]
        
            BRANCHES = [
                'Zamalek', 'New Cairo', 'Maadi', 'Heliopolis', 'Nasr City', 
                'Dokki', 'Sheikh Zayed', '6th October', 'Mohandessin', 'Garden City',
                'Downtown Cairo', 'Agouza', 'Rehab', 'Katameya', '5th Settlement',
                'Korba', 'Sheraton', 'Madinet Nasr', 'El Tagamoa', 'October Plaza',
                'Mall of Arabia', 'City Stars', 'Cairo Festival City', 'Americana Plaza'
            ]

            stores = []
            start_id = 200
            increment = 10

            selected_names = random.sample(STORE_NAMES, n)
            
            for i in range(n):
                store = {
                    'store_id': start_id + (i * increment),
                    'store_name': selected_names[i],
                    'branch': random.choice(BRANCHES),
                    'average_bags_at_9AM': random.randint(5, 30),
                    'average_overall_rating': round(random.uniform(1.0, 5.0), 1),
                    'price': round(random.uniform(15.0, 75.0), 2),
                    'longitude': round(random.uniform(31.00, 31.60), 4),
                    'latitude': round(random.uniform(29.90, 30.20), 4)
                }
                stores.append(store)

                
            with open(save_file, 'w', newline='') as file:
                fields = ['store_id', 'store_name', 'branch', 'average_bags_at_9AM', 
                        'average_overall_rating', 'price', 'longitude', 'latitude']
                
                writer = csv.DictWriter(file, fieldnames = fields)
                writer.writeheader()
                writer.writerows(stores)
            
            print(f"Generated {n} stores to {save_file}")
            return [store['store_id'] for store in stores]
        
        except Exception as e:
            print(f"Error generating stores: {e}")



    def _customer_generator(self, n, save_file, store_ids):
        try:
            customers = []

            for i in range(1, n + 1):
                customer = {
                    'customer_id': i,
                    'longitude': round(random.uniform(31.00, 31.60), 4),
                    'latitude': round(random.uniform(29.90, 30.20), 4)
                }
                

                for store_id in store_ids:
                    # valuation = round(random.uniform(1.0, 5.0), 1)

                    # 20% chance customer has never tried this store
                    if random.random() < 0.2:
                        valuation = 0.0
                    else:
                        valuation = round(random.uniform(1.0, 5.0), 1) # else rate from 1 to 5

                    customer[f'store{store_id}_valuation'] = valuation
                    
                customers.append(customer)

            fields = ['customer_id', 'longitude', 'latitude'] + \
                    [f'store{store_id}_valuation' for store_id in store_ids]
            
            with open(save_file, 'w', newline = '') as file:
                writer = csv.DictWriter(file, fieldnames = fields)
                writer.writeheader()
                writer.writerows(customers)
            
            print(f"Generated {n} customers to {save_file}")
        
        except Exception as e:
            print(f"Error generating customers: {e}")

    def _generate_filepaths(self, directory): # find the next file names for stores and customers to generate data
        try:
            folder = directory
            prefix_stores = "stores"
            prefix_customers = "customers"
            extension = ".csv"

            if not os.path.exists(folder):
                os.makedirs(folder)

            last_num = 1

            # scan all files inside datasets/
            for filename in os.listdir(folder):
                if filename.startswith(prefix_stores) and filename.endswith(extension):
                    number = filename[len(prefix_stores):-len(extension)]

                    if int(number) > last_num:
                        last_num = int(number)

            # next dataset number is +1
            next_num = last_num + 1

            # finally get file paths
            stores_path = os.path.join(folder, f"{prefix_stores}{str(next_num)}.csv")
            customers_path = os.path.join(folder, f"{prefix_customers}{str(next_num)}.csv")

            return stores_path, customers_path
        except Exception as e:
            print(f"Error generating filepath: {e}")
    
    # MAIN FUNCTION HEREEEEEEEEEEEEEEEEEE
    def generate(self, store_count = 15, customer_count = 100, min_est_bags = 5, max_est_bags = 30, save_directory = "datasets"):
        try:
            # first we find the next file names to be written to by the data genrators
            # then we generate stores
            # then we generate customers but we need to pass the store_ids generated to get valuations
            store_path, customer_path = self._generate_filepaths(save_directory)

            store_ids = self._store_generator(store_count, store_path)
            self._customer_generator(customer_count, customer_path, store_ids)
        except Exception as e:
            print(f"Error in data generator: {e}")


# call the function
generator = DataGenerator()
generator.generate()


Generated 15 stores to datasets\stores8.csv
Generated 100 customers to datasets\customers8.csv


In [43]:
class Simulator:
    # all the generated customers and all the generated stores are in the simulation everyday
    # IMPROVEMENT LATER: not all customers open the app everyday, select only a portion of the customers in the first day and each day randomly add a random number of customers and exclude some
    # ie. not all the customers open the app every single day
    def __init__(self, stores_file, customers_file):
        self.stores = None 
        self.customers = None 
        self.curr_day = 1

        self.sim_results = {
            "days": {}
        }

    def _simulate_actual_bags(self):
        results = {} 

        for store in self.stores:
            # TODO: for each store simulate the number of actual bags
            # dont forget to update the store object
            pass 

        return results
         

    def _simulate_reservations(self): 
        store_reserved_count = 0
        customer_reservations = { "customer_id": None, "reserved_store_id": None}
        
        for customer in self.customers:
            # TODO: reservation logic
            # simulate each customer's reservaiton, update their reserved store, and increment that store's reservations
            pass

        return store_reserved_count, customer_reservations


    def _simulate_sales(self): 
        # simulate which bags were actually sold and which were cancelled
        # get reserved and actual for each store -> sold is either reserved or actual 
        # update each store's  sold and cancelld and actual and reserved
        # update relevant data OF curr day in self.sim_results
        pass 

    def _simulate_customer_valuation_updates(self, day_customers):
        # update the customer's valuation for the store they reserved at (IF ANY)
        # based on whether they got the bag (random from 1.0-5.0) or if it got cancelled (0.0)
        # update relevant data OF curr day in self.day_results
        pass 


    def simulate(self, stores, customers, days = 10):
        # loop for days and update the sim_results (stores data for each day for all the days)

        self.all_stores = stores
        self.all_customers = customers 
        self.curr_day = 1

        # TODO: 
        # keep in mind the customer arrival order especially if actual < estimated
        # so the first x customers get their bags and the later customers get cancelled

        for day in range(1, days + 1):
            self.curr_day = day
            day_result = {
                "stores": {},
                "customers": {}
            }

            for store in stores:
                day_result["stores"][store.get_store_id()] = {
                    "estimated": store.get_estimated_bags(),
                    "actual": None,
                    "reserved": 0,
                    "sold": None,
                    "cancelled": None
                }

            for customer in customers:
                day_result["customers"][customer.get_customer_id()] = {
                    "reserved_store": None,
                    "got_bag": None,
                    "new_rating": None
                }

            self._simulate_reservations()
            self._simulate_actual_bags()
            self._simulate_sales()
            self._simulate_customer_valuation_updates()

        return self.sim_results()


In [44]:
def data_reader(stores_filepath, customers_filepath):
    stores = []
    customers = []

    # read stores file
    # fields: 
    # store_id,store_name,branch,average_bags_at_9AM,average_overall_rating,price,longitude,latitude
    # create 1 store object for each row and append to stores array
    # init: store_id, store_name, branch, estimated_bags, rating, longitude, latitude
    with open(stores_filepath, "r") as f:
        reader = csv.DictReader(f)

        for row in reader:
            store = Store(
                store_id = int(row["store_id"]),
                store_name = row["store_name"],
                branch = row["branch"],
                estimated_bags = int(row["average_bags_at_9AM"]),
                rating = float(row["average_overall_rating"]),
                # price = float(row["price"]),
                longitude = float(row["longitude"]),
                latitude = float(row["latitude"])
            )
            store.set_price_per_bag(float(row["price"])) # this is assuming the price will change from day to day
            stores.append(store)


    # read customers file
    # fields: 
    # customer_id,longitude,latitude,store100_valuation,store200_valuation,store300_valuation,store400_valuation,store500_valuation,store600_valuation,store700_valuation,store800_valuation,store900_valuation,store1000_valuation
    # create 1 customer object for each row and append to customers array
    # init: customer_id, valuations, longitude, latitude
    with open(customers_filepath, "r") as f:
        reader = csv.DictReader(f)
        for row in reader:
            # Extract all valuation columns
            # valuations = {
            #     key: float(value)
            #     for key, value in row.items()
            #     if key.startswith("store") and key.endswith("_valuation")
            # }
            valuations =  {
                int(key.replace("store", "").replace("_valuation", "")): float(value)
                for key, value in row.items()
                if key.startswith("store") and key.endswith("_valuation")
            }

            customer = Customer(
                customer_id = int(row["customer_id"]),
                valuations = valuations,
                longitude = float(row["longitude"]),
                latitude = float(row["latitude"]),
            )

            customers.append(customer)

    return stores, customers


In [45]:
def basic_simulation(stores, customers):
    """
    basic simulation for milestone 2
    - uses Marketplace with the greedy baseline
    - at the end we compute KPIs from Marketplace
    """
    try:
        app = Marketplace(stores=stores, customers=customers)

        print("=== SIMULATION START ===\n")

        # each customer arrives, sees some stores, and reserves from one of them
        for customer in app.get_customers():
            # print(f"Customer {customer.get_customer_id()} views the following stores:")

            selected_stores = app.select(customer)

            # for i, store in enumerate(selected_stores, start=1):
            #     print(f"{i}: {store.get_store_name()}")

            # simulate a reservation choice
            if selected_stores:   # make sure list is not empty
                chosen_store = random.choice(selected_stores)

                if chosen_store.get_unreserved_bags() > 0:
                    chosen_store.reserve_bag()
                    customer.reserve(chosen_store.get_store_id())

            print()

        print("=== END OF DAY SUMMARY ===\n")
        for store in app.get_stores():
            print(f"{store.get_store_name()} | Reserved: {store._bags_reserved}/{store.get_estimated_bags()}")

        print("\nCustomer reservations:")
        for customer in app.get_customers():
            reserved_store = customer.get_reserved_store()
            if reserved_store is None:
                print(f"Customer {customer.get_customer_id()} FAILED to reserve a store")
            else:
                print(f"Customer {customer.get_customer_id()} reserved from store: {reserved_store}")


        kpis = app.compute_kpis()

        print("\n=== KPIs FOR GREEDY BASELINE (MILESTONE 2) ===")
        print(f"Total bags sold:          {kpis['total_bags_sold']}")
        print(f"Total bags unsold:        {kpis['total_bags_unsold']}")
        print(f"Total bags canceled:      {kpis['total_bags_canceled']}")
        print(f"Total revenue:            {kpis['total_revenue']:.2f}")
        print(f"Max possible revenue:     {kpis['max_possible_revenue']:.2f}")
        print(f"Revenue lost:             {kpis['revenue_lost']:.2f}")
        print(f"Cancellation rate:        {kpis['cancellation_rate']:.3f}")
        print(f"Food waste rate:          {kpis['food_waste_rate']:.3f}")
        print(f"Revenue efficiency:       {kpis['revenue_efficiency']:.3f}")
        print(f"Customers who left:       {kpis['customers_left']}")
        print(f"Fairness score (0–1):     {kpis['fairness_score']:.3f}")


        print("\n=== SIMULATION END ===")

    except Exception as e:
        print(f"Error Running Simulation: {e}")

In [49]:
# TODO: actual waste
# TODO: complete Simulator class, move most of the main logic to Simulator class
# once simulator class is complete, basic_simulation funciton will be deprecated (but keep it in the code)

# 0 demand --> actual waste from store
# 5 bags 1 customer --> customer gets all that food...not waste from tbs but waste from customer

def main():
    # here is where we run the simulation
    
    stores_filepath = "datasets/stores6.csv"
    customers_filepath = "datasets/customers6.csv"

    # data_reader returns lists of Store and Customer objects
    stores, customers = data_reader(stores_filepath, customers_filepath)

    basic_simulation(stores, customers)

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"Error running program: {e}")


=== SIMULATION START ===





































































































=== END OF DAY SUMMARY ===

Dunkin | Reserved: 10/18
Momochi | Reserved: 0/17
Piano Piano | Reserved: 6/19
Blue Bottle | Reserved: 0/23
Stiletto | Reserved: 8/8
Tim Hortons | Reserved: 9/26
Cairo Kitchen | Reserved: 12/21
La Poire | Reserved: 0/28
Greco | Reserved: 9/17
Bird Cage | Reserved: 7/19
Espresso House | Reserved: 8/14
Arabica | Reserved: 10/26
Abou El Sid | Reserved: 5/5
Nero Caffè | Reserved: 8/12
Beano's | Reserved: 8/19

Customer reservations:
Customer 1 reserved from store: 320
Customer 2 reserved from store: 280
Customer 3 reserved from store: 250
Customer 4 reserved from store: 310
Customer 5 reserved from store: 320
Customer 6 reserved from store: 340
Customer 7 reserved from store: 260
Customer 8 reserved from store: 260
Customer 9 reserved from store: 280
Customer 10 reserved from store: 330
Customer 11 reserved from store: 320
Customer 12 res